<a href="https://colab.research.google.com/github/nxvya/MAINPROJECT/blob/main/CNN%2BTCN%2BLSTM%2BATTENTION%20NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.models import Model
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, concatenate
from keras.layers import TimeDistributed, LSTM
from keras.optimizers import Adam

In [ ]:
import pandas as pd
# Replace 'your_dataset.csv' with the
data = pd.read_csv('PhiUSIIL_Phishing_URL_Dataset.csv')

In [ ]:
X = data.drop(columns=['label'])
y = data['label']

In [ ]:
label_encoders = {}
for column in X.select_dtypes(include='object'):
    label_encoders[column] = LabelEncoder()
    X[column] = label_encoders[column].fit_transform(X[column])



In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Standardize numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.select_dtypes(include=['int64', 'float64']))
X_test_scaled = scaler.transform(X_test.select_dtypes(include=['int64', 'float64']))


In [ ]:

input_shape = (X_train_scaled.shape[1],)  # Number of features

In [ ]:
!pip install tensorflow

In [ ]:
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D

In [ ]:
!pip install tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# CNN layer
cnn_input = Input(shape=(None, 10))
cnn_layer = Conv1D(filters=64, kernel_size=3, activation='relu')(cnn_input)
cnn_layer = MaxPooling1D(pool_size=2)(cnn_layer)

In [ ]:

from tensorflow.keras.layers import Input, TimeDistributed, Dense
# TCN layer
tcn_input = Input(shape=(1, 55))
tcn_layer = TimeDistributed(Dense(64, activation='relu'))(tcn_input)

In [ ]:
from tensorflow.keras.layers import Bidirectional, LSTM, Input
lstm_input = Input(shape=(1, 55))
lstm_layer = Bidirectional(LSTM(64, return_sequences=True))(lstm_input)

In [ ]:
# Import the Reshape layer
from keras.layers import Reshape

# Reshape LSTM output to match dimensions
lstm_reshaped = Reshape((-1, 64))(lstm_layer)

In [ ]:
print("cnn_layer shape:", cnn_layer.shape)
print("tcn_layer shape:", tcn_layer.shape)
print("lstm_reshaped shape:", lstm_reshaped.shape)

cnn_layer shape: (None, None, 64)
tcn_layer shape: (None, 1, 64)
lstm_reshaped shape: (None, 2, 64)


In [ ]:
!pip install -q tensorflow
import tensorflow as tf

In [ ]:
cnn_layer = tf.keras.layers.Lambda(lambda x: tf.keras.backend.reshape(x, (-1, 1, x.shape[-1])))(cnn_layer)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Reshape
print("CNN Layer Shape:", cnn_layer.shape)
print("TCN Layer Shape:", tcn_layer.shape)
print("LSTM Reshaped Layer Shape:", lstm_reshaped.shape)


CNN Layer Shape: (None, 1, 64)
TCN Layer Shape: (None, 1, 64)
LSTM Reshaped Layer Shape: (None, 2, 64)


In [ ]:
cnn_layer = tf.keras.layers.Reshape((1, cnn_layer.shape[-1]))(cnn_layer)
tcn_layer = tf.keras.layers.Reshape((1, tcn_layer.shape[-1]))(tcn_layer)

In [ ]:
import tensorflow as tf

# Assuming lstm_reshaped has shape (None, 2, 64)
lstm_reshaped = tf.keras.layers.Reshape((1, -1))(lstm_reshaped)


In [ ]:
from tensorflow.keras.layers import Input, Dense, Concatenate, Conv1D, MaxPooling1D, Flatten, Dropout, LSTM, Bidirectional, Attention

In [ ]:
cnn_layer_reshaped = Dense(128, activation="relu")(cnn_layer)
attention = Attention()([cnn_layer_reshaped, tcn_layer, lstm_reshaped])

In [ ]:
# Merge layers
merged_layer = concatenate([cnn_layer, tcn_layer, lstm_reshaped, attention])
merged_layer = Flatten()(merged_layer)


In [ ]:
# Fully connected layer
fc_layer = Dense(64, activation='relu')(merged_layer)

In [ ]:
# Output layer
output_layer = Dense(1, activation='sigmoid')(fc_layer)

In [ ]:
# Create model
model = Model(inputs=[cnn_input, tcn_input, lstm_input], outputs=output_layer)


In [ ]:

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
print(model.input_shape)

[(None, None, 10), (None, 1, 55), (None, 1, 55)]


In [ ]:
print(X_train_scaled.shape)

(188636, 55)


In [ ]:
X_train_scaled = X_train_scaled.reshape((-1, 1, 55))

In [ ]:
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(None, 55)),
    # ...
])

In [ ]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",metrics=['accuracy']
)

In [ ]:
model.fit(X_train_scaled, y_train, epochs=10, batch_size=64, validation_split=0.2)


Epoch 1/10
2358/2358 [==============================] - 15s 6ms/step - loss: 7.0652 - accuracy: 0.0871 - val_loss: 7.0660 - val_accuracy: 0.0854
Epoch 2/10
2358/2358 [==============================] - 18s 7ms/step - loss: 7.0660 - accuracy: 0.0871 - val_loss: 7.0660 - val_accuracy: 0.0854
Epoch 3/10
2358/2358 [==============================] - 12s 5ms/step - loss: 7.0656 - accuracy: 0.0871 - val_loss: 7.0660 - val_accuracy: 0.0854
Epoch 4/10
2358/2358 [==============================] - 9s 4ms/step - loss: 7.0649 - accuracy: 0.0871 - val_loss: 7.0660 - val_accuracy: 0.0854
Epoch 5/10
2358/2358 [==============================] - 10s 4ms/step - loss: 7.0645 - accuracy: 0.0871 - val_loss: 7.0660 - val_accuracy: 0.0854
Epoch 6/10
2358/2358 [==============================] - 9s 4ms/step - loss: 7.0653 - accuracy: 0.0871 - val_loss: 7.0660 - val_accuracy: 0.0854
Epoch 7/10
2358/2358 [==============================] - 10s 4ms/step - loss: 7.0653 - accuracy: 0.0871 - val_loss: 7.0660 - val_accu

In [ ]:
# Step 4: Evaluate the model
loss, accuracy = model.evaluate([X_test_scaled], y_test)
print(f'Test Accuracy: {accuracy}')

1474/1474 [==============================] - 2s 1ms/step - loss: 6.0942 - accuracy: 0.0837
Test Accuracy: 0.08365317434072495


In [ ]:
model.save('url_malicious_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
 #Step 5: Make predictions
def predict_malicious(url_features):
    scaled_features = scaler.transform(url_features)
    prediction = model.predict([scaled_features, scaled_features, scaled_features])
    if prediction >= 0.5:
        return "Malicious"
    else:
        return "Not Malicious"

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
max_len = 100
def preprocess_url(url):
    # Tokenize and pad the URL
    url_seq = tokenizer.texts_to_sequences([url])
    url_padded = pad_sequences(url_seq, maxlen=max_len)
    return url_padded

def predict_url_maliciousness(url):
    preprocessed_url = preprocess_url(url)
    prediction = model.predict(preprocessed_url)[0][0]
    if prediction <= 0.5:
        return "Malicious"
    else:
        return "Not Malicious"

# Example usage:
user_url = input("Enter a URL to check if it's malicious: ")
result = predict_url_maliciousness(user_url)
print(f"The URL '{user_url}' is predicted to be: {result}")


Enter a URL to check if it's malicious: wwww.google.com


1/1 [==============================] - 0s 83ms/step
The URL 'wwww.google.com' is predicted to be: Not Malicious


In [ ]:
model.save('url_malicious_model.h5')
